In [ ]:
#@title Notebook Number
notebook_number=1 #@param {type:'number'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Path for Articles (PDF Formet Only)
path = "/content/drive/MyDrive/<Fill this In>" # @param {type:"string"}

In [ ]:
%cd $path

/content/drive/MyDrive/Digital Twin Project/Article Checker/Final


In [ ]:
%pip install PyPDF2 typing_extensions anthropic pytesseract pdf2image xlsxwriter easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 19.1 MB/s eta 0:00:00


In [ ]:
!sudo apt install tesseract-ocr -y
# Install Poppler
!apt-get install -y poppler-utils -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,606 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
!dir

Automate_GPT_10.ipynb	       gpt_final_3.xlsx
Automate_GPT_11.ipynb	       gpt_final_4.xlsx
Automate_GPT_12.ipynb	       gpt_final_5.xlsx
Automate_GPT_13.ipynb	       gpt_final_6.xlsx
Automate_GPT_14.ipynb	       gpt_final_7.xlsx
Automate_GPT_15.ipynb	       gpt_final_8.xlsx
Automate_GPT_1.ipynb	       gpt_final_9.xlsx
Automate_GPT_2.ipynb	       gpt_final_chunks_10.xlsx
Automate_GPT_3.ipynb	       gpt_final_chunks_11.xlsx
Automate_GPT_4.ipynb	       gpt_final_chunks_12.xlsx
Automate_GPT_5.ipynb	       gpt_final_chunks_13.xlsx
Automate_GPT_6.ipynb	       gpt_final_chunks_14.xlsx
Automate_GPT_7.ipynb	       gpt_final_chunks_15.xlsx
Automate_GPT_8.ipynb	       gpt_final_chunks_1.xlsx
Automate_GPT_9.ipynb	       gpt_final_chunks_2.xlsx
Automate_GPT.ipynb	       gpt_final_chunks_3.xlsx
batch_input_1.jsonl	       gpt_final_chunks_4.xlsx
Bernard\ 2024.pdf	       gpt_final_chunks_5.xlsx
binary_mentions_df1.csv        gpt_final_chunks_6.xlsx
binary_mentions_df2.csv        gpt_final_chunks_7.xl

In [ ]:
#Enter OpenAI API Key Here
api_key = "" # @param {type:"string")

In [ ]:
!pip install openai==0.28.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [ ]:
import os
from pathlib import Path
import PyPDF2
import openai
import pandas as pd
import shutil
import time
from openai.error import RateLimitError

# Assuming api_key and path are defined elsewhere
# openai.api_key = api_key
# directory = path

models = ["gpt-4o-mini", "gpt-4o", "o1-mini"]  # List of models to run

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text

def extract_title_with_gpt(client, text, model):
    retries = 3
    messages = [
        {
            "role": "user",
            "content": f"Extract the title from the following article text: {text[:500]}"
        }
    ]
    for attempt in range(retries):
        try:
            response = client.ChatCompletion.create(
                model=model,
                messages=messages
            )
            title = response['choices'][0]['message']['content'].strip()
            if len(title) <= 150:
                return title
        except RateLimitError:
            print(f"Rate limit reached during title extraction. Retrying after delay.")
            time.sleep(300)
        except Exception as e:
            print(e)
    return "Unknown"

def extract_author_with_gpt(client, text, model):
    retries = 3
    messages = [
        {
            "role": "user",
            "content": f"Extract the last name of the main author from the following article text followed by et al. (e.g. 'Sanan et al.'): {text[:500]}"
        }
    ]
    for attempt in range(retries):
        try:
            response = client.ChatCompletion.create(
                model=model,
                messages=messages
            )
            author = response['choices'][0]['message']['content'].strip()
            if len(author) <= 50:
                return author
            print(f"Retrying author extraction due to length (Attempt {attempt + 1})...")
        except RateLimitError:
            print(f"Rate limit reached during author extraction. Retrying after delay.")
            time.sleep(1800)
        except Exception as e:
            print(e)
    return "Unknown"

def extract_industry_with_gpt(client, text, model):
    retries = 3
    messages = [
        {
            "role": "user",
            "content": f"Tell the industry name that the article focuses on (e.g. 'Water', 'Energy', etc.). If there is none write 'General': {text[:1500]}"
        }
    ]
    for attempt in range(retries):
        try:
            response = client.ChatCompletion.create(
                model=model,
                messages=messages
            )
            industry = response['choices'][0]['message']['content'].strip()
            if len(industry) <= 50:
                return industry
            print(f"Retrying industry extraction due to length (Attempt {attempt + 1})...")
        except RateLimitError:
            print(f"Rate limit reached during industry extraction. Retrying after delay.")
            time.sleep(1800)
        except Exception as e:
            print(e)
    return "General"

def explain_parameter_presence(client, text, parameter, definition, model):
    retries = 3
    messages = [
        {
            "role": "user",
            "content": (
                f"Explain whether the parameter '{parameter}' is mentioned/talked about in the following text "
                f"and provide evidence from the text. If it does, briefly explain how (3-5 sentences with ~2 pieces of evidence); "
                f"if it does not match, briefly explain why the paper does not focus on it (1 sentence). "
                f"Note that '{parameter}' is defined as '{definition}'. {text}"
            )
        }
    ]
    for attempt in range(retries):
        try:
            response = client.ChatCompletion.create(
                model=model,
                messages=messages
            )
            explanation = response['choices'][0]['message']['content'].strip()
            return explanation
        except RateLimitError:
            print(f"Rate limit reached during parameter explanation. Retrying after delay.")
            time.sleep(1800)
        except Exception as e:
            print(e)
    return "Explanation not available"

# Parameters and their definitions
parameters = [
    "Physical Entity and Processes", "Virtual Entity", "Virtual Processes",
    "Physical Environment", "Virtual Environment", "Realization", "State",
    "Metrology", "Fidelity", "Parameters", "Twinning Rate",
    "Use Cases", "Perceived Benefits", "Data Ownership", "Scope",
    "P2V Connection", "V2P Connection"
]

definitions = [
    "Physical Entity and Processes: A tangible object or a real-world process within a physical system, such as a piece of equipment, a component, or an operational process in an industrial setting such as a manufacturing operation.",
    "Virtual Entity: A digital counterpart or replica of a physical entity or process within a digital twin environment. It mimics the behavior, characteristics, and attributes of its physical counterpart. It is usually represented as a digital model.",
    "Virtual Processes: A simulation or emulation of a real-world process within a digital environment. It involves replicating the behavior, interactions, and outcomes of the physical process using computational models and algorithms. It can include optimization, prediction, simulation, and analysis, among others.",
    "Physical Environment: Refers to the actual physical surroundings or context in which a system operates, including physical infrastructure, equipment, facilities, and natural elements.",
    "Virtual Environment: Any number of virtual ‘worlds’ or simulations that replicate the state of the physical environment and designed for specific use-case(s), e.g., health monitoring, production schedule optimization.",
    "Realization: How the delta between the physical and digital entities is determined and the virtual or physical entity is updated accordingly. The impact of the measured changes in one entity on the other.",
    "State: The current value of the parameters of either the physical or virtual entity/environment.",
    "Metrology: Refers to measurement of a state, including the methods, standards, and instruments used to quantify physical properties or attributes in the virtual and physical systems (how data is collected). Example sensors (used to record data), flow meters, or a thermometer (measuring the temperature of a physical motor).",
    "Fidelity: Degree of accuracy and completeness with which a digital twin replicates the physical counterpart of the system, related to data, behavior, physical structure, etc. Examples include “comprehensive physical and functional description” or fully mirroring the characteristics and functionality (of a physical entity).",
    "Parameters: Refers to the variables, attributes, or inputs that define and characterize the behavior, performance, and state of a system within a digital twin. Parameters can include physical properties, environmental conditions, operational settings, and performance metrics, e.g., flow rate, water quality, pressure, temperature, permeate flow, wear, flux, etc. These can be measured values or constants tuned through model training.",
    "Twinning Rate: The frequency or time interval with which the entities are synchronized, and information flow is updated, such as 'real-time', every second, minute, 15 minutes, 1 hour, etc.",
    "Use-Cases: The applications of the Digital Twin, e.g., decision support, simulation, forecasting.",
    "Perceived Benefits: The envisaged advantages achieved in realizing the Digital Twin, e.g., improved design, behavior, structure, manufacturability, conformance, etc.",
    "Data Ownership: The legal ownership and security of the data stored within the Digital Twin.",
    "Scope: The breadth of the purview of the digital twin, i.e., what portion of the physical system the virtual entity represents. The system components or subsystems within the range of the digital twin.",
    "P2V Connection: Means by which the state of the physical entity is transferred to the virtual environment, including IoT sensors, SCADA, web services, 5G, or other communication protocols.",
    "V2P Connection: The reciprocal connection or interaction between the virtual and physical components of a digital twin, enabling feedback, control, and adaptation of physical systems based on insights, simulations, or analyses conducted in the virtual environment. Examples of this in practice include changes in display terminals, PLCs, process control machine parameters, and production management."
]

# Initialize OpenAI client
openai.api_key = api_key

directory = path
checked_folder_path = Path(directory) / 'Checked'
checked_folder_path.mkdir(exist_ok=True)

# Process for each model
for model in models:
    print(f"Processing with model: {model}")
    # Initialize DataFrame for explanations for this model
    df_parameter_explanations = pd.DataFrame()

    # Initialize counter for articles
    article_count = 0

    # Process each PDF file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            article_count += 1  # Increment the article counter
            file_path = Path(directory) / filename
            pdf_text = extract_text_from_pdf(file_path)
            if pdf_text:
                title = extract_title_with_gpt(openai, pdf_text, model=model)
                author = extract_author_with_gpt(openai, pdf_text, model=model)
                industry = extract_industry_with_gpt(openai, pdf_text, model=model)
                explanation_data = {'Title': title, 'Author': author, 'Industry': industry}

                # Initialize counter for parameters
                parameter_count = 0

                # Get explanations for each parameter if mentioned in the text
                for i, parameter in enumerate(parameters):
                    definition = definitions[i]
                    while True:
                        try:
                            explanation = explain_parameter_presence(openai, pdf_text, parameter, definition, model=model)
                            explanation_data[parameter] = explanation
                            parameter_count += 1  # Increment the parameter counter
                            print(f"Article {article_count} - Parameter '{parameter}': Done")
                            break  # If successful, break out of the loop
                        except RateLimitError as e:
                            print(f"Rate limit reached for Article {article_count} - Parameter '{parameter}'. Retrying soon.")
                            time.sleep(1800)
                        except Exception as e:
                            print(f"Error processing parameter '{parameter}' for article {article_count}: {e}")
                            break

                # Append explanations to the DataFrame
                df_parameter_explanations = pd.concat([df_parameter_explanations, pd.DataFrame([explanation_data])], ignore_index=True)

                # Optionally move processed file to 'Checked' folder. Note this will interfere if there's multiple models.
                # shutil.move(os.path.join(directory, filename), os.path.join(checked_folder_path, filename))

                print(f"Article {article_count}: Done")

    # Save the DataFrame to a CSV file
    output_csv_path = Path(directory) / f'parameter_explanations_{model}.csv'
    df_parameter_explanations.to_csv(output_csv_path, index=False)

Processing with model: gpt-4o-mini
Article 1 - Parameter 'Physical Entity and Processes': Done
Article 1 - Parameter 'Virtual Entity': Done
Article 1 - Parameter 'Virtual Processes': Done
Article 1 - Parameter 'Physical Environment': Done
Article 1 - Parameter 'Virtual Environment': Done
Article 1 - Parameter 'Realization': Done
Article 1 - Parameter 'State': Done
Article 1 - Parameter 'Metrology': Done
Article 1 - Parameter 'Fidelity': Done
Article 1 - Parameter 'Parameters': Done
Article 1 - Parameter 'Twinning Rate': Done
Article 1 - Parameter 'Use Cases': Done
Article 1 - Parameter 'Perceived Benefits': Done
Article 1 - Parameter 'Data Ownership': Done
Article 1 - Parameter 'Scope': Done
Article 1 - Parameter 'P2V Connection': Done
Article 1 - Parameter 'V2P Connection': Done
Article 1: Done
Article 2 - Parameter 'Physical Entity and Processes': Done
Article 2 - Parameter 'Virtual Entity': Done
Article 2 - Parameter 'Virtual Processes': Done
Article 2 - Parameter 'Physical Environm

KeyboardInterrupt: 

In [ ]:
df_parameter_explanations.head()

In [ ]:
import pandas as pd
from collections import Counter
import time
from pathlib import Path
from openai.error import RateLimitError
import openai

# Assuming api_key, path, models, parameters, definitions, and df_parameter_explanations are defined elsewhere

# Initialize OpenAI client
openai.api_key = api_key
client = openai  # Using the openai module directly as the client

def check_parameter_mentioned(client, explanation):
    """
    Determines whether a parameter is mentioned in the explanation using GPT-4o.

    Args:
        client: OpenAI client.
        explanation (str): The explanation text to analyze.

    Returns:
        int: 1 if the parameter is mentioned, 0 otherwise.
    """
    max_attempts = 3
    results = []
    model = "gpt-4o"  # Always use gpt-4o for binary checking

    messages = [
        {
            "role": "system",
            "content": "Indicate with a 1 if the explanation explicitly mentions the parameter; otherwise, return 0."
        },
        {
            "role": "user",
            "content": explanation[:500]
        }
    ]

    for attempt in range(max_attempts):
        try:
            response = client.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=0,
                max_tokens=5,
                top_p=1
            )
            answer = response.choices[0].message.content.strip()

            # Validate if answer is either '1' or '0'
            if answer in ['1', '0']:
                results.append(answer)
                break  # Exit loop after first valid response

        except RateLimitError:
            print(f"Rate limit reached during parameter check. Retrying after delay.")
            time.sleep(1800)  # Wait for 30 minutes before retrying
        except Exception as e:
            print(f"Error during parameter check: {e}")
            break  # Exit on non-rate limit exceptions

    # Determine the most frequent result or default to '0'
    if results:
        most_frequent_result = Counter(results).most_common(1)[0][0]
    else:
        most_frequent_result = '0'  # Default if no valid results

    return int(most_frequent_result)

# Directory where the CSV files are located
directory = Path(path)

# Iterate over each model to process corresponding CSV files
for model in models:
    print(f"\nProcessing binary indicators for model: {model}")

    # Define the input and output CSV file paths
    input_csv_path = directory / f'parameter_explanations_{model}.csv'
    output_csv_path = directory / f'parameters_binary_{model}.csv'

    # Check if the input CSV exists
    if not input_csv_path.exists():
        print(f"Input CSV file {input_csv_path} does not exist. Skipping this model.")
        continue

    # Read the parameter explanations CSV
    try:
        df_explanations = pd.read_csv(input_csv_path)
        print(f"Loaded {len(df_explanations)} articles from {input_csv_path.name}")
    except Exception as e:
        print(f"Error reading {input_csv_path.name}: {e}")
        continue

    # Initialize a new DataFrame for binary indicators
    df_binary = pd.DataFrame(columns=['Title', 'Author', 'Industry'] + parameters)

    # Initialize counter for processed articles
    article_count = 0

    # Iterate over each row (article) in the explanations DataFrame
    for index, row in df_explanations.iterrows():
        binary_data = {
            'Title': row.get('Title', ""),
            'Author': row.get('Author', ""),
            'Industry': row.get('Industry', "")
        }

        for parameter in parameters:
            explanation = row.get(parameter, "")
            if pd.isna(explanation):
                explanation = ""

            # Check if the parameter is mentioned using GPT-4o
            binary_flag = check_parameter_mentioned(client, explanation)
            binary_data[parameter] = binary_flag

        # Append the binary data to the DataFrame
        df_binary = pd.concat([df_binary, pd.DataFrame([binary_data])], ignore_index=True)

        article_count += 1
        if article_count % 10 == 0 or article_count == len(df_explanations):
            print(f"Processed {article_count}/{len(df_explanations)} articles for model {model}")

    # Save the binary indicators DataFrame to a new CSV file
    try:
        df_binary.to_csv(output_csv_path, index=False)
        print(f"Saved binary indicators to {output_csv_path.name}")
    except Exception as e:
        print(f"Error saving {output_csv_path.name}: {e}")

print("\nAll models have been processed.")

In [ ]:
df_binary.head()

In [ ]:
import pandas as pd
from pathlib import Path

output_file = f"gpt_final_{notebook_number}.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    for model in models:
        # Define the input CSV file paths for each model
        explanations_file = directory / f"parameter_explanations_{model}.csv"
        binary_file = directory / f"parameters_binary_{model}.csv"

        # Load and write explanations data to a separate sheet
        if explanations_file.exists():
            try:
                df_explanations = pd.read_csv(explanations_file)
                df_explanations.to_excel(writer, sheet_name=f"Explanations_{model}", index=False)
            except Exception as e:
                print(f"Error reading or writing {explanations_file.name}: {e}")

        # Load and write binary data to a separate sheet
        if binary_file.exists():
            try:
                df_binary = pd.read_csv(binary_file)
                df_binary.to_excel(writer, sheet_name=f"Binary_{model}", index=False)
            except Exception as e:
                print(f"Error reading or writing {binary_file.name}: {e}")

print(f"All DataFrames saved to: {output_file}")

In [ ]:
import os
from pathlib import Path
import pytesseract
from PIL import Image
import pdf2image

def ocr_from_pdf(pdf_path):
    # Convert PDF to a list of images
    images = pdf2image.convert_from_path(pdf_path)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image) + "\n"
    return text

directory = path
articles = []

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        file_path = Path(directory) / filename
        pdf_text = ocr_from_pdf(file_path)
        articles.append(pdf_text)

def chunk_generator(text, chunk_size):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield " ".join(words[i:i + chunk_size])

chunk_size = 500  # Number of words per chunk, adjust as needed
chunks = tuple()

for article in articles:
    article_chunks = tuple(chunk_generator(article, chunk_size))
    chunks = chunks + (article_chunks,)

In [ ]:
import re

# Function to clean OCR text
def clean_ocr_text(raw_text):
    # Remove erroneous newline characters and unwanted symbols
    cleaned_text = re.sub(r'\n', ' ', raw_text)  # Replace newlines with space
    cleaned_text = re.sub(r'[@“”\|]', '', cleaned_text)  # Remove specific unwanted characters
    cleaned_text = re.sub(r' – ', '-', cleaned_text)  # Correct spaced hyphens
    # Replace incorrect newlines (e.g., those within words)
    cleaned_text = re.sub(r'(?<=[a-zA-Z])-(?=[a-zA-Z])', '', cleaned_text)
    # Remove any other post-processing steps you need here
    return cleaned_text

# Use the function on the OCR result
articles = [clean_ocr_text(text) for text in articles]

In [ ]:
import time
import pandas as pd
from tqdm import tqdm  # For progress bar
from openai.error import RateLimitError

# Placeholder for predefined variables:
# client (OpenAI client), models, parameters, definitions, chunks, and articles

# Initialize an empty dictionary to store DataFrames for each model
model_dfs = {}

# Process each model
for model in models:
    print(f"Processing model: {model}")
    df_parameter_explanations = pd.DataFrame()  # Create a new DataFrame for each model

    # Initialize a single progress bar for the entire processing loop
    total_iterations = len(articles) * len(parameters) * sum(len(ch) for ch in chunks)
    progress_bar = tqdm(total=total_iterations, desc="Processing Articles and Parameters")

    # Loop through articles and chunks
    article_count = 0

    for article_chunks, full_article_text in zip(chunks, articles):
        article_count += 1

        # Extract metadata for the article
        title = extract_title_with_gpt(client, full_article_text, model)
        author = extract_author_with_gpt(client, full_article_text, model)
        industry = extract_industry_with_gpt(client, full_article_text, model)

        explanation_data = {'Title': title, 'Author': author, 'Industry': industry}

        # Process each parameter
        for i, parameter in enumerate(parameters):
            definition = definitions[i]
            parameter_explanations = []

            # Process each chunk in the article for the current parameter
            for chunk_count, chunk in enumerate(article_chunks, start=1):
                try:
                    # Update progress bar with the current status
                    progress_bar.set_description(f"Article {article_count} - Parameter '{parameter}' - Chunk {chunk_count}")

                    # Get explanation using GPT-4 model
                    explanation = explain_parameter_presence(client, chunk, parameter, definition, model)
                    parameter_explanations.append(explanation)

                except RateLimitError:
                    print(f"Rate limit reached for Article {article_count} - Parameter '{parameter}'. Retrying soon.")
                    time.sleep(1800)  # Wait 30 minutes before retrying

                # Update the progress bar by 1 step
                progress_bar.update(1)

            # Store explanations in the explanation data dictionary
            explanation_data[parameter] = tuple(parameter_explanations)

        # Append explanation data to the DataFrame for the current model
        df_parameter_explanations = pd.concat([df_parameter_explanations, pd.DataFrame([explanation_data])], ignore_index=True)

    # Close the progress bar after processing the model
    progress_bar.close()

    # Save the DataFrame for the current model to the dictionaryhttps://www.w3schools.com/cpp/
    model_dfs[model] = df_parameter_explanations
    print(f"Model {model} processing complete.")

# `model_dfs` now holds a DataFrame for each model with processed explanations
print("All models processed.")

Processing model: gpt-4o-mini



Processing Articles and Parameters:   0%|          | 0/24310 [1:03:52<?, ?it/s]

Article 1 - Parameter 'Physical Entity and Processes' - Chunk 1:   0%|          | 0/24310 [00:01<?, ?it/s]
Article 1 - Parameter 'Physical Entity and Processes' - Chunk 1:   0%|          | 1/24310 [00:04<30:45:14,  4.55s/it]
Article 1 - Parameter 'Physical Entity and Processes' - Chunk 2:   0%|          | 1/24310 [00:04<30:45:14,  4.55s/it]
Article 1 - Parameter 'Physical Entity and Processes' - Chunk 2:   0%|          | 2/24310 [00:07<24:22:01,  3.61s/it]
Article 1 - Parameter 'Physical Entity and Processes' - Chunk 3:   0%|          | 2/24310 [00:07<24:22:01,  3.61s/it]
Article 1 - Parameter 'Physical Entity and Processes' - Chunk 3:   0%|          | 3/24310 [00:10<21:00:00,  3.11s/it]
Article 1 - Parameter 'Physical Entity and Processes' - Chunk 4:   0%|          | 3/24310 [00:10<21:00:00,  3.11s/it]
Article 1 - Parameter 'Physical Entity and Processes' - Chunk 4:   0%|          | 4/24310 [00:12<19:37:

Retrying author extraction due to length (Attempt 1)...
Retrying author extraction due to length (Attempt 2)...
Retrying author extraction due to length (Attempt 3)...



Article 10 - Parameter 'Physical Entity and Processes' - Chunk 1:   9%|▉         | 2278/24310 [1:20:01<9:32:41,  1.56s/it]
Article 10 - Parameter 'Physical Entity and Processes' - Chunk 1:   9%|▉         | 2279/24310 [1:20:03<18:14:50,  2.98s/it]
Article 10 - Parameter 'Physical Entity and Processes' - Chunk 2:   9%|▉         | 2279/24310 [1:20:03<18:14:50,  2.98s/it]
Article 10 - Parameter 'Physical Entity and Processes' - Chunk 2:   9%|▉         | 2280/24310 [1:20:06<17:45:44,  2.90s/it]
Article 10 - Parameter 'Physical Entity and Processes' - Chunk 3:   9%|▉         | 2280/24310 [1:20:06<17:45:44,  2.90s/it]
Article 10 - Parameter 'Physical Entity and Processes' - Chunk 3:   9%|▉         | 2281/24310 [1:20:09<17:19:10,  2.83s/it]
Article 10 - Parameter 'Physical Entity and Processes' - Chunk 4:   9%|▉         | 2281/24310 [1:20:09<17:19:10,  2.83s/it]
Article 10 - Parameter 'Physical Entity and Processes' - Chunk 4:   9%|▉         | 2282/24310 [1:20:11<16:00:30,  2.62s/it]
Article 

Model gpt-4o-mini processing complete.
Processing model: gpt-4o


...

Retrying author extraction due to length (Attempt 1)...


Article 9 - Parameter 'V2P Connection' - Chunk 20:   9%|▉         | 2278/24310 [2:17:00<30:28:19,  4.98s/it]

Retrying author extraction due to length (Attempt 1)...
Retrying author extraction due to length (Attempt 2)...
Retrying author extraction due to length (Attempt 3)...


Article 10 - Parameter 'V2P Connection' - Chunk 9:  10%|█         | 2431/24310 [2:26:16<21:56:31,  3.61s/it]

Model o1-mini processing complete.
All models processed.


In [ ]:
model_dfs["gpt-4o-mini"]

,Title,Author,Industry,Physical Entity and Processes,Virtual Entity,Virtual Processes,Physical Environment,Virtual Environment,Realization,State,Metrology,Fidelity,Parameters,Twinning Rate,Use Cases,Perceived Benefits,Data Ownership,Scope,P2V Connection,V2P Connection
0,"The title of the article is ""Embrace Digital T...",Corriveau et al.,Water,(The parameter 'Physical Entity and Processes'...,"(The parameter ""Virtual Entity"" is not explici...",(The parameter 'Virtual Processes' is not expl...,(The parameter 'Physical Environment' is not e...,(The parameter 'Virtual Environment' is not sp...,(The parameter 'Realization' is not explicitly...,(The parameter 'State' is mentioned in the tex...,(The parameter 'Metrology' is not explicitly m...,(The parameter 'Fidelity' is not explicitly me...,(The parameter 'Parameters' is not explicitly ...,(The parameter 'Twinning Rate' is not explicit...,(The parameter 'Use Cases' is not explicitly m...,(The parameter 'Perceived Benefits' is discuss...,(The parameter 'Data Ownership' is not explici...,(The parameter 'Scope' is not directly mention...,(The parameter 'P2V Connection' is not explici...,(The parameter 'V2P Connection' is not explici...
1,"The title extracted from the article text is ""...",DeJong et al.,Construction,(The parameter 'Physical Entity and Processes'...,(The parameter 'Virtual Entity' is not explici...,(The parameter 'Virtual Processes' is not expl...,(The parameter 'Physical Environment' is menti...,(The parameter 'Virtual Environment' is not ex...,(The parameter 'Realization' is not explicitly...,(The parameter 'State' is indirectly addressed...,(The parameter 'Metrology' is not explicitly m...,(The parameter 'Fidelity' is not explicitly me...,(The parameter 'Parameters' is not explicitly ...,(The parameter 'Twinning Rate' is not explicit...,(The parameter 'Use Cases' is indeed mentioned...,(The parameter 'Perceived Benefits' is indirec...,(The parameter 'Data Ownership' is not specifi...,(The parameter 'Scope' is not explicitly menti...,(The parameter 'P2V Connection' is indeed addr...,(The parameter 'V2P Connection' is not explici...
2,"The title of the article is ""Digital Twin-Base...",Zhou et al.,Water,"(The parameter ""Physical Entity and Processes""...",(The parameter 'Virtual Entity' is not explici...,"(The parameter ""Virtual Processes"" is indirect...",(The parameter 'Physical Environment' is not e...,(The parameter 'Virtual Environment' is not ex...,(The parameter 'Realization' is not explicitly...,(The parameter 'State' is referenced in the te...,(The parameter 'Metrology' is not explicitly m...,(The parameter 'Fidelity' is not explicitly me...,(The parameter 'Parameters' is not explicitly ...,(The parameter 'Twinning Rate' is not explicit...,(The parameter 'Use Cases' is not explicitly m...,(The parameter 'Perceived Benefits' is present...,(The parameter 'Data Ownership' is not explici...,(The parameter 'Scope' is not explicitly menti...,(The parameter 'P2V Connection' is not explici...,(The parameter 'V2P Connection' is not explici...
3,"The title of the article is ""Planning the rest...",Rooij et al.,Desalination,(The parameter 'Physical Entity and Processes'...,"(Yes, the parameter 'Virtual Entity' is mentio...",(The parameter 'Virtual Processes' is indeed m...,(The parameter 'Physical Environment' is indir...,(The parameter 'Virtual Environment' is effect...,(The parameter 'Realization' is not explicitly...,(The parameter 'State' is indeed mentioned in ...,(The parameter 'Metrology' is not explicitly m...,(The parameter 'Fidelity' is not explicitly me...,(The parameter 'Parameters' is indeed mentione...,(The parameter 'Twinning Rate' is not mentione...,(The parameter 'Use Cases' is indeed reference...,(The parameter 'Perceived Benefits' is indeed ...,(The parameter 'Data Ownership' is not explici...,(The parameter 'Scope' is discussed in the tex...,(The parameter 'P2V Connection' is not explici...,(The parameter 'V2P Connection' is not explici...
4,"

In [ ]:
import pandas as pd
import time
from openai.error import RateLimitError
from collections import defaultdict
from tqdm import tqdm  # For single progress bar
import openai

# Placeholder for predefined variables and lists, such as models, parameters, and model_dfs (a dictionary of DataFrames keyed by model names)

def check_parameter_mentioned(title, parameter, explanation, model="gpt-4"):
    """Checks if a parameter is mentioned in the explanation using the OpenAI API."""
    messages = [
        {"role": "system", "content": "Indicate with a 1 if the explanation says the parameter is mentioned, else return 0:"},
        {"role": "user", "content": explanation}
    ]
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=0,
                max_tokens=5
            )
            # Extract the response content, expected to be '1' or '0'
            return response['choices'][0]['message']['content'].strip()
        except RateLimitError:
            print(f"Rate limit reached for parameter '{parameter}' in title '{title}'. Retrying after delay.")
            time.sleep(60)  # Wait before retrying
        except Exception as e:
            print(f"Error processing parameter '{parameter}' in title '{title}': {e}")
            break  # Exit on unknown error
    return '0'  # Default to '0' if all retries fail

# Dictionary to store binary DataFrames for each model
model_binary_results = {}

# Calculate the total number of steps for the progress bar
total_steps = sum(len(df) * len(parameters) for df in model_dfs.values())

# Initialize a single progress bar for all processing
progress_bar = tqdm(total=total_steps, desc="Processing Models", unit="step")

# Process each model DataFrame in model_dfs (assumed to be a dictionary of DataFrames)
for model, model_df in model_dfs.items():
    print(f"Processing model: {model}")
    df_parameters_binary = pd.DataFrame(columns=['Title', 'Author', 'Industry'] + parameters)

    results = defaultdict(lambda: '0')  # Store results with default '0'

    for index, row in model_df.iterrows():
        title = row['Title']
        binary_data = {'Title': title, 'Author': row['Author'], 'Industry': row['Industry']}

        for parameter in parameters:
            explanations = row[parameter]  # Tuple with explanations for each chunk
            mention_count = 0

            # Update the progress bar description dynamically
            progress_bar.set_description(f"Model: {model}, Article: {index + 1}, Parameter: {parameter}")

            for explanation in explanations:
                # Check if the parameter is mentioned in the chunk
                result = check_parameter_mentioned(title, parameter, explanation, model)
                if result == '1':
                    mention_count += 1

                # Stop checking further if already met the threshold
                if mention_count >= 2:
                    break

            # Set binary flag based on threshold
            binary_data[parameter] = 1 if mention_count >= 2 else 0

            # Update the progress bar by one step for each parameter processed
            progress_bar.update(1)

        # Append the binary data to the DataFrame
        df_parameters_binary = pd.concat([df_parameters_binary, pd.DataFrame([binary_data])], ignore_index=True)

    # Store the binary results DataFrame for the current model
    model_binary_results[model] = df_parameters_binary
    print(f"Model {model} processing completed.")

# Close the progress bar after all processing is complete
progress_bar.close()

# All models processed. Now, model_binary_results holds binary DataFrames for each model in memory.

Model: gpt-4o-mini, Article: 1, Parameter: Physical Entity and Processes:   0%|          | 0/510 [00:00<?, ?step/s]

Processing model: gpt-4o-mini


Model: gpt-4o, Article: 1, Parameter: Physical Entity and Processes:  33%|███▎      | 170/510 [10:54<13:51,  2.45s/step]

Model gpt-4o-mini processing completed.
Processing model: gpt-4o


Model: o1-mini, Article: 1, Parameter: Physical Entity and Processes:  67%|██████▋   | 340/510 [14:35<04:26,  1.57s/step]

...

Error processing parameter 'Scope' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processing parameter 'P2V Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processing parameter 'P2V Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processing parameter 'P2V Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processing parameter 'P2V Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processing paramet

Model: o1-mini, Article: 10, Parameter: V2P Connection: 100%|█████████▉| 509/510 [26:32<00:02,  2.61s/step]

Error processing parameter 'P2V Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processing parameter 'V2P Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processing parameter 'V2P Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processing parameter 'V2P Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processing parameter 'V2P Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Error processin

Model: o1-mini, Article: 10, Parameter: V2P Connection: 100%|██████████| 510/510 [26:35<00:00,  3.13s/step]

Error processing parameter 'V2P Connection' in title '**Using Digital Twins to Improve Pumping and Distribution**': Unsupported value: 'messages[0].role' does not support 'system' with this model.
Model o1-mini processing completed.


In [ ]:
df_parameters_binary

In [ ]:
import pandas as pd

# Define the output file name
output_file = f"gpt_final_chunks_{notebook_number}.xlsx"

# Dictionary `model_binary_results` contains the binary results DataFrame for each model
# Dictionary `model_dfs` contains the original explanation DataFrame for each model

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    for model in models:
        # Save the binary DataFrame for each model to a sheet named "Binary_<model_name>"
        df_parameters_binary = model_binary_results.get(model)
        if df_parameters_binary is not None:
            df_parameters_binary.to_excel(writer, sheet_name=f"Binary_{model}", index=False)

        # Save the explanation DataFrame for each model to a sheet named "Explanations_<model_name>"
        df_parameter_explanations = model_dfs.get(model)
        if df_parameter_explanations is not None:
            df_parameter_explanations.to_excel(writer, sheet_name=f"Explanations_{model}", index=False)

print(f"All DataFrames saved to: {output_file}")

All DataFrames saved to: gpt_final_chunks_1.xlsx
